## 13단계: 가변 길이 인수(역전파 편)

> 이번 단계에서는 가변 길이 인수에 대응하는 역전파를 구현합니다.

### 13.1 가변 길이 인수에 대응한 Add 클래스의 역전파

`그림 13-1`과 같이 리스트나 튜플을 거치지 않고 인수와 결과를 직접 주고받도록 개선해보자.

<img src="images/그림 13-1.png" width=600/>

위 그림을 수식으로 확인하면, $y = x_0 + x_1$일 때 미분하면 $\frac{\partial{y}}{\partial{x_0}} = 1$, $\frac{\partial{y}}{\partial{x_1}} = 1$이 된다.

In [5]:
# 필요 모듈 정의

import numpy as np

class Variable:
    def __init__(self, data):
        if not isinstance(data, (type(None), np.ndarray)):
            raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.asarray(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
    
    def backward(self, gys):
        raise NotImplementedError()

In [6]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        # 덧셈의 역전파는 입력이 1개, 출력이 2개이다.
        # 이 부분의 대응을 위해 Variable 클래스를 수정해야 한다.
        return gy, gy

### 13.2 Variable 클래스 수정

Add 클래스와 같이 입력과 출력이 여러 개인 함수의 backward 메서드 대응을 위해 Variable 클래스를 수정한다.

In [7]:
class Variable:
    def __init__(self, data):
        if not isinstance(data, (type(None), np.ndarray)):
            raise TypeError(f'{type(data)}은(는) 지원하지 않습니다.')
        
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            
            # 기존 코드
            # x, y = f.input, f.output  # 함수의 입출력이 1개임을 가정한다.
            # x.grad = f.backward(y.grad)  # backward 메서드를 호출한다.
            
            # if x.creator is not None:
            #     funcs.append(x.creator)
            
            # 수정 코드
            gys = [output.grad for output in f.outputs]  # 1. 출력변수의 미분값들을 리스트에 담는다.
            gxs = f.backward(*gys)  # 2. backward 메서드에 미분값들을 언팩(unpack)하여 넣어준다.
            if not isinstance(gxs, tuple):  # 3. 만약 gxs가 튜플이 아니라면 튜플로 변환한다.
                gxs = (gxs,)
            
            for x, gx in zip(f.inputs, gxs):  # 4.backward로 생성된 미분값을 입력변수의 grad에 저장한다.
                # f.inputs[i]와 gxs[i]는 서로 대응 관계에 있다.
                x.grad = gx

                if x.creator is not None:
                    funcs.append(x.creator)
                    

Q. 위 구현은 입력변수의 순서와 backward 메서드의 결과값의 순서가 동일함이 보장되어야 한다. \
이를 Function 인터페이스에서 자연스럽게 강제할 수 있도록 해야할까?

### 13.3 Square 클래스 구현

Function 클래스의 입력변수(*inputs)가 복수형(tuple)으로 변경되었으므로 해당 부분만 수정해준다.

In [10]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        # 기존 코드
        # x = self.input.data
        
        # 수정 코드
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def add(x0, x1):
    return Add()(x0, x1)

def square(x):
    return Square()(x)

x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


보다시피 $z = x^2 + y^2$라는 계산의 순전파와 역전파가 정상 작동됨을 확인했다.\
이상으로 복수의 입출력에 대응한 자동 미분 구조를 완성했다. \
그러나 지금의 DeZero에는 몇 가지 문제가 있는데, 다음 단계에서는 이 문제들을 해결하겠다.